# CS202: Compiler Construction

## In-class Exercises, Week of 04/11/2022

----

In [2]:
from cs202_support.eval_x86 import X86Emulator

init = """
  .globl main
main:
  movq $16384, %rdi
  movq $16, %rsi
  callq initialize
  movq rootstack_begin(%rip), %r15
"""

----

# Part 1: select-instructions

## Question 1

Write x86 instructions corresponding to the output of `select-instructions` on the following `assign` statement:

```
Assign("x", Allocate(24, (int, int)))
```

1. Store the current `free_ptr` in `x` as the location of the allocated tuple
2. Increment `free_prt` by the size of the tuple
3. Store the tuple's tag in its first location
    - Move the tuple into %r11
    - Move the tag into 0(%r11)

```
movq free_ptr(%rip), #x
addq $24, free_ptr(%rip)
movq #x, %r11
movq $3, 0(%r11)
```

In [3]:
asm = init + """
movq free_ptr(%rip), #x
addq $24, free_ptr(%rip)
movq #x, %r11
movq $3, 0(%r11)
"""

emu = X86Emulator(logging=False)
emu.eval_program(asm)
emu.print_state()

,Location,Value
0,mem 100000,3
1,reg rbp,1000
2,reg rsp,1000
3,reg rdi,16384
4,reg rsi,16
5,reg r15,2000
6,reg r11,100000
7,var x,100000
8,main,FunPointer(fun_name='main')
9,rootstack_begin,2000


## Question 2

Write x86 instructions corresponding to the output of `select-instructions` on the following statement:

```
Collect(16)
```

1. Move the pointer to the root stack into %rdi
2. Move the umber of bytes to collect into $rsi
3. Call the `collect` function

```
movq %r15, %rdi
movq $16, %rsi
callq collect
```

## Question 3

Write x86 instructions corresponding to the output of `select-instructions` on the following statements:

a. `Assign("y", Subscript("x", 0))`

b. `Assign(Subscript("x", 0), ConstantInt(1))`

a.
1. Move the tuple into %r11
2. Move the value at offset 8*(i+1) into the target variable

```
movq #x, %r11
movq 8(%r11), #y
```

b.
1. Move the tuple into %r11
2. Move the value we want to write into offset 8*(i+1) relative to r11

```
movq #x, %r11
movq $1, 8(%r11)
```


In [7]:
asm = init + """
movq free_ptr(%rip), #x
addq $24, free_ptr(%rip)
movq #x, %r11
movq $3, 0(%r11)
movq #x, %r11
movq $1, 8(%r11)
movq #x, %r11
movq $2, 16(%r11)
movq #x, %r11
movq 8(%r11), #y
"""

emu = X86Emulator(logging=False)
emu.eval_program(asm)
emu.print_state()

,Location,Value
0,mem 100000,3
1,mem 100008,1
2,mem 100016,2
3,reg rbp,1000
4,reg rsp,1000
5,reg rdi,16384
6,reg rsi,16
7,reg r15,2000
8,reg r11,100000
9,var x,100000


----

# Part 2: register allocation

## Question 4

Why do we need to ensure that tupled-valued variables are spilled when we call the collector? How can we accomplish this?

1. The collector looks for tuple-valued variables on the root stack, so all tuple-valued variables need to be spilled there in order for the collector to work.
2. We will use our interference graph to make sure that tuple-valued variables live after a all to the collector interfere with *all registers*, so that the register allocator will force those variables into root stack locations.

- interference graph will now contain registers
- Register allocator will need to deal with registers int the interference graph

## Question 5

How can we modify register allocation to ensure that tuple-valued variables get spilled to the root stack, and non-tuple-valued variables get spilled to the regular stack?

build-interference:
- When you find a `callq collect` instruction:
    - For each variable in the live-after set:
        - If it is a tuple valued variable(VecVar):
            - Add an edge between the variable and each register
- For all `callq` instructions(including to collect), handle caller-saved registers:
    - For each variable in the live-after set:
        - Add and edge between the variable and each caller-saved register

allocate-registers:
- Need to make two major changes in the pass:
    1. Change the graph coloring algorithm to pre-assign colors to register before coloring
    2. In building `homes`, we need to put vars on the regular stack and tuple vars on  the root stack (or in registers
- Part 1:
    1. Pre-assign colors for all the registers
    2. Pre-populate saturation sets for variables based on those colors
    3. Color the variables in the  graph as normal
- Part 2:
    - Old version: single `color_mao: color -> loc`
    - New version: two color maps
        - `color_map: color -> loc` (registers + stack locations)
        - `tuple_color_map: color -> loc` (registers + root stack locations)
    - To populate the maps:
        - First add all register color to both maps
        - Then, loop over variables `x`
            - If `x` is a tuple var:
                - If `x` has a register color, do nothing
                - Else, create a root stack location and assign `x`'s color to that location and increment the number of root stack locations used
            - If `x` is a regular var:
                - Same thing but with the regular stack

----

# Part 3: print-x86

## Question 6

Write x86 assembly for the `main` block of the program:

```
x = (1, 2)
```

We need to add boilerplate code for:
- Initializing heap
- Setting up root stack

The new code looks like:
```
movq $16348, %rdi                   -- set the total allowed size of the root stack in bytes
movq $16, %rsi                      -- set the total size of the heap
callq initizlize                    -- sets up t he root stack and heap
movq rootstack_begin(%rip), %r15    -- puts a pointer to the root stack in %r15

```

## Question 7

What additional code is needed in `main` if the root stack is non-empty? What is this code needed for? Consider e.g. the program:

```
x = (1, 2)
y = (3, 4)
print(x[0])
```

```
movq $16384, $rdi
movq $16, %rsi
callq initizalize
movq rootstack_begin(%rip), %r15
movq $0, 0(%r15)                    -- ensure that the root stack location we will use in initialized to 0
addq $8, %r15                       -- increment the root stack pointer for the location we have used
```

The final two instructions are repeated once for each stack location used

We initialize each location to 0 so the garbage collector knows that are not yet used

In [ ]:
asm = """
YOUR SOLUTION HERE
"""

emu = X86Emulator(logging=False)
emu.eval_program(asm)
emu.print_state()